In [ ]:
! pip install gradio
# Install the library using pip
!pip install sentence-transformers
! pip install sent2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.6 MB/s eta 0:

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

#model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [ ]:
import numpy as np
from sent2vec.vectorizer import Vectorizer
from scipy import spatial
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import requests
from bs4 import BeautifulSoup
import gradio as gr

In [ ]:
def get_hotel_recommendations(query):
    # Define the URL
    url = "https://www.booking.com/placestostay/region/th/krabi.th.html?aid=377400&label=krabi-Sy33n2ehDwUxGwtUTgdxDwS43018971241%3Apl%3Ata%3Ap11040%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-6466156461%3Alp9073357%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YdbYVqXDN8zp7PNDFvT66M8&sid=b7852b7029329594b266443d9e155981&lang=th&soz=1&lang_click=other&cdl=en-gb&lang_changed=1"

    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all elements with the class name that contains hotel data
        hotel_data = soup.find_all(class_="sr__card js-sr-card")

        # Create a list to store hotel data as dictionaries
        hotel_list = []

        # Extract and store the hotel names, prices, score badges, and other information
        for data in hotel_data:
            hotel_name = data.find("span",class_="bui-card__title")
            hotel_name_text = hotel_name.text.strip() if hotel_name else "N/A"

            hotel_price = data.find("div",class_="bui-price-display__value bui-f-color-constructive")
            hotel_price_text = hotel_price.text.strip() if hotel_price else "N/A"

            score_badge = data.find("div",class_="bui-review-score__badge")
            score_badge_text = score_badge.text.strip() if score_badge else "N/A"


            hotel_location = data.find("span", itemprop="addressLocality")
            hotel_location_text = hotel_location.text.strip() if hotel_location else "N/A"

            # Create a dictionary to store the hotel data
            hotel = {
                "Name": hotel_name_text,
                "Price": hotel_price_text,
                "Score Badge": score_badge_text,
                "Location": hotel_location_text,

            }

            # Append the hotel dictionary to the list
            hotel_list.append(hotel)

        df = pd.DataFrame(hotel_list)

        # Sort the DataFrame by Score Badge in descending order
        df = df.sort_values(by="Score Badge", ascending=False)

        # Reset the DataFrame index
        df.reset_index(drop=True, inplace=True)

        return df


    else:
        return "Failed to retrieve the web page."

In [ ]:
data_hotel = get_hotel_recommendations("Krabi")
data_hotel

,Name,Price,Score Badge,Location
0,Cha Cha Bungalow,"THB 1,200",9.8,เกาะจำ
1,Aonang Eco Villa - SHA Extra Plus,"THB 3,290",9.6,หาดอ่าวนาง
2,The Mantra Navi,"THB 1,900",9.4,"Long Beach, เกาะลันตา"
3,Aonang Inn,"THB 1,000",9.2,เมืองกระบี่
4,JR Place,"THB 1,247.80",9.2,หาดคลองม่วง
5,Lanta Dareen Garden Home,THB 270,9.1,"Mai Kaew, เกาะลันตา"
6,Tinidee Hideaway Tonsai Beach Krabi - SHA Extr...,"THB 4,180",9.1,หาดต้นไทร
7,The Lazy Lodge,"THB 2,500",9.1,"Klong Khong Beach, เกาะลันตา"
8,The Brown Hotel,"THB 1,760",9,เมืองกระบี่
9,Poonsiri Varich Pool Villa Aonang,"THB 6,560",9,หาดอ่าวนาง


In [ ]:
intown_filter = data_hotel[data_hotel['Location'].str.contains('เมืองกระบี่')]

# Print the filtered result
intown_filter

,Name,Price,Score Badge,Location
3,Aonang Inn,"THB 1,000",9.2,เมืองกระบี่
8,The Brown Hotel,"THB 1,760",9,เมืองกระบี่


In [ ]:
lanta_filter = data_hotel[data_hotel['Location'].str.contains('เกาะลันตา')]

# Print the filtered result
lanta_filter

,Name,Price,Score Badge,Location
2,The Mantra Navi,"THB 1,900",9.4,"Long Beach, เกาะลันตา"
5,Lanta Dareen Garden Home,THB 270,9.1,"Mai Kaew, เกาะลันตา"
7,The Lazy Lodge,"THB 2,500",9.1,"Klong Khong Beach, เกาะลันตา"


In [ ]:
lanta_filter = data_hotel[data_hotel['Location'].str.contains('หาดอ่าวนาง')]

# Print the filtered result
lanta_filter

,Name,Price,Score Badge,Location
1,Aonang Eco Villa - SHA Extra Plus,"THB 3,290",9.6,หาดอ่าวนาง
9,Poonsiri Varich Pool Villa Aonang,"THB 6,560",9,หาดอ่าวนาง


In [ ]:
klong_filter = data_hotel[data_hotel['Location'].str.contains('หาดคลองม่วง')]

# Print the filtered result
klong_filter

,Name,Price,Score Badge,Location
4,JR Place,"THB 1,247.80",9.2,หาดคลองม่วง


In [ ]:
tonsai_filter = data_hotel[data_hotel['Location'].str.contains('หาดต้นไทร')]

# Print the filtered result
tonsai_filter

,Name,Price,Score Badge,Location
6,Tinidee Hideaway Tonsai Beach Krabi - SHA Extr...,"THB 4,180",9.1,หาดต้นไทร


In [ ]:
# Assuming you have a DataFrame named 'filtered_result'
formatted_strings = []

for _, row in lanta_filter.iterrows():
    name = f"Name: {row['Name']}"
    price = f"Price: {row['Price']}"
    score = f"Score Badge: {row['Score Badge']}"
    location = f"Location: {row['Location']}"

    formatted_strings.append('\n'.join([name, price, score, location]))

# Join the formatted strings with newline characters to create a multi-line string
result_string = '\n\n'.join(formatted_strings)

# Print the resulting multi-line string
print(result_string)


Name: Aonang Eco Villa - SHA Extra Plus
Price: THB 3,290
Score Badge: 9.6
Location: หาดอ่าวนาง

Name: Poonsiri Varich Pool Villa Aonang
Price: THB 6,560
Score Badge: 9
Location: หาดอ่าวนาง


In [ ]:
# Create a Gradio interface for the chatbot

iface = gr.Interface(
      fn=(get_hotel_recommendations),
      inputs="text",
      outputs="text",
      layout="vertical",
      title="Hotel Recommendations Chatbot",
      description="Enter a query to get the top 5 hotel recommendations in Krabi.",
  )
  # Launch the Gradio interface
iface.launch()



<ipython-input-12-0cb5a5969666>:3: GradioDeprecationWarning: `layout` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9767fce50e722f95e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
